In [2]:
# example script to generate boosted predictions
import csv
from tqdm import tqdm
from datasets import load_dataset
from cb import CB
from util import accuracy

# load dataset
lambada = load_dataset("EleutherAI/lambada_openai")
X = lambada["test"]["text"]

# boosting params
alphas = [-0.6, -0.5, -0.5, -0.5]
ks = [10, 11, 10, 9]
models = ["gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"]

accs = []
for i, model in enumerate(tqdm(models)):
    alpha, k = alphas[i], ks[i]
    cb_model = CB(alpha, k, model_id=model, device="cuda")
    out = cb_model.boosted_batched_generate(X, fmax_score=True, batch_size=64)
    acc = accuracy(out["targets"], out["preds_fmax"])
    acc_boost = accuracy(out["targets"], out["preds_cb"])
    accs.append({"model": model, "fmax": acc, "cb": acc_boost})

print(accs)

Generating test split:   0%|          | 0/5153 [00:00<?, ? examples/s]

Dataset lambada_openai downloaded and prepared to /root/.cache/huggingface/datasets/EleutherAI___lambada_openai/default/1.0.0/57baddecfa09d1790541ef07274c5666abfbe9d2ccd0cd46013cd557b0343095. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]


  0%|          | 0/4 [00:00<?, ?it/s]

Map:   0%|          | 0/5153 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/5153 [00:00<?, ? examples/s]

100%|██████████| 81/81 [02:04<00:00,  1.54s/it]

 75%|███████▌  | 3/4 [05:02<01:55, 115.85s/it]

Map:   0%|          | 0/5153 [00:00<?, ? examples/s]

100%|██████████| 81/81 [04:01<00:00,  2.99s/it]

100%|██████████| 4/4 [10:04<00:00, 151.13s/it]

[{'model': 'gpt2', 'fmax': tensor(0.4667), 'cb': tensor(0.6470)}, {'model': 'gpt2-medium', 'fmax': tensor(0.5500), 'cb': tensor(0.7141)}, {'model': 'gpt2-large', 'fmax': tensor(0.5876), 'cb': tensor(0.7413)}, {'model': 'gpt2-xl', 'fmax': tensor(0.6142), 'cb': tensor(0.7475)}]


In [16]:
file_name = "boosted_lambada.csv"
with open(file_name, "w") as f:
    w = csv.DictWriter(f, accs[0].keys())
    w.writeheader()
    for row in accs:
        values = [i[1] for i in row.items()][1:]
        out = {'model': row['model'], 'fmax': values[0].item(), 'cb': values[1].item()}
        w.writerow(out)